In [86]:
import os
import pandas as pd

# 폴더 경로
folder_path = 'C:/Users/smhrd/Desktop/부동산'

# 폴더 내 모든 .csv 파일 경로 리스트 생성
csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

# 모든 csv 파일을 읽어서 DataFrame 리스트 생성
df_list = [pd.read_csv(file) for file in csv_files]

# DataFrame 모두 합치기
df_all = pd.concat(df_list, ignore_index=True)


In [87]:
df_all = df_all.drop(columns=['현재업종', '준공인가일', '권리금', '매물특징', '입주가능일', '해당층/총층'], errors='ignore')

In [88]:
df_all['소재지'] = df_all['소재지'].str.replace('광주시 서구 ', '', regex=False)

In [ ]:
df_all[df_all['추천업종'] == "-"]
df_all[df_all['추천업종'].isna()]

In [90]:
df_all['추천업종'] = df_all['추천업종'].fillna('-')

In [ ]:
df_all['소재지'].unique()

In [92]:
df_all = df_all[df_all['소재지'] != '광주시 남구 월산동']

In [ ]:
df_all['소재지'].unique()

In [94]:
df_all['소재지'] = df_all['소재지'].replace({
    '유촌동': '유덕동',  # ❌ → 이건 반대
    '내방동': '화정동',  # ❌ → 이것도 반대
    '마륵동': '서창동',  # ✅ ← 이것만 맞음
    '매월동': '서창동'   # ✅
})

In [ ]:
sorted_unique_values = sorted(df_all['소재지'].unique())
print(sorted_unique_values)

In [ ]:
df_all[df_all['소재지']=="쌍촌동"]

In [99]:
import numpy as np

# 기준선: (위도, 경도) 순서로 두 점 정의 (예시값, 실제 경계에 맞게 수정 필요)
lat1, lon1 = 35.150003489023504, 126.8559765815735  # 기준선의 첫 번째 점
lat2, lon2 = 35.15224919419664,126.87131881713869  # 기준선의 두 번째 점

# 기준선의 직선식 계산 (Ax + By + C = 0)
A = lat2 - lat1
B = lon1 - lon2
C = lat1 * lon2 - lat2 * lon1

# 직선 기준으로 점이 위쪽(>0)이면 상무1동, 아래쪽(<0)이면 상무2동
def classify_sangmu(lat, lon):
    result = A * lon + B * lat + C
    return '상무1동' if result > 0 else '상무2동'

# 쌍촌동 데이터에 적용
mask = df_all['소재지'] == '쌍촌동'
df_all.loc[mask, '소재지'] = df_all.loc[mask].apply(
    lambda row: classify_sangmu(row['위도'], row['경도']), axis=1
)


In [ ]:
import json
import folium

# 1. JSON 파일 열기
with open('C:/Users/smhrd/Desktop/seogu.geojson', 'r', encoding='utf-8') as f:
    geo_data = json.load(f)

# 4. folium 지도 생성
m = folium.Map(
    location=[35.1445, 126.9156],
    tiles='cartodbpositron',
    zoom_start=13,
    zoom_control=True,          # 줌 버튼 비활성화
    scrollWheelZoom=True,       # 마우스 휠 줌 비활성화
    dragging=True               # 드래그 이동 비활성화
)

# 5. GeoJson 추가 (툴팁에 동 이름 표시)
folium.GeoJson(
    geo_data,
    name='광주 서구 통합 동',
    tooltip=folium.GeoJsonTooltip(fields=['adm_nm'], aliases=['행정동:'])
).add_to(m)

# ['광천동', '금호동', '농성동', '동천동', '상무1동', '상무2동', '서창동', '양동', '유덕동', '치평동', '풍암동', '화정동']
sangmu1_data = df_all[df_all['소재지'] == '화정동']

for _, row in sangmu1_data.iterrows():
# for _, row in df_all.iterrows():
    latitude = row['위도']
    longitude = row['경도']
    
    # 마커 추가
    folium.Marker(
        location=[latitude, longitude],
        popup=f"{row['소재지']}",  # 클릭 시 표시될 텍스트
        icon=folium.Icon(color='blue')  # 마커 색상 (파란색)
    ).add_to(m)

# 7. LayerControl 추가 (GeoJson 레이어를 켜고 끌 수 있는 버튼)
folium.LayerControl().add_to(m)

# 8. 지도 출력
m


In [ ]:
# '(전용률'이 포함되지 않은 행 필터링
# filtered_df = df_all[~df_all['계약/전용면적'].astype(str).str.contains('\(전용률')]
filtered_df = df_all[~df_all['계약/전용면적'].astype(str).str.contains(r'\(전용률')]

# 결과 출력 (원하는 컬럼만 보려면 예: ['계약/전용면적'])
print(filtered_df['계약/전용면적'].unique())
df_all['계약/전용면적'] = df_all['계약/전용면적'].fillna('-')
df_all[df_all['계약/전용면적'] != '-']

In [ ]:
import re
df_all['계약/전용면적'] = df_all['계약/전용면적'].apply(
    lambda x: re.sub(r'\(.*?\)', '', x).strip() if pd.notna(x) and x != '-' else x
)
df_all

In [ ]:
df_all['월세'] = df_all['월세'].astype(str).str.replace(r'^월세', '', regex=True).str.strip()
df_all

In [134]:
df_all.to_json('부동산.json', orient='records', force_ascii=False, indent=2)